In [1]:
import keras
from keras.preprocessing import sequence
import numpy as np
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow import set_random_seed

# compute accuracy without mekeras method
from sklearn.metrics import accuracy_score

# fix random seed for reproducibility
import random

/home/jayati/anaconda3/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [3]:
import numpy as np
from keras.datasets import mnist

# read training data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_trainset = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255

y_trainset = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

X_trainset = X_trainset[:6000]
y_trainset = y_trainset[:6000]

#test data
test_size = 100
idx = np.random.choice(range(len(X_test)), size=test_size, replace=False)
X_test = X_test[idx]
y_test = y_test[idx]

train_size = 2000
idx = np.random.choice(range(len(X_trainset)), size=train_size, replace=False)

batch_size = X_train.shape[0]/5
num_epochs = 1
step = 200

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier


def mymodel():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    
    return model

In [5]:
model = mymodel()



history = model.fit(X_trainset[idx], y_trainset[idx], 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    verbose=1, 
                    callbacks=None, validation_split=0.0, 
                    validation_data=None, shuffle=True, 
                    class_weight=None, sample_weight=None, 
                    initial_epoch=0, steps_per_epoch=None,
                    validation_steps=None)

pred = model.predict(x=X_test)
y_pred = [ elem.argmax() for elem in pred]
y_true = [ elem.argmax() for elem in y_test]

print( "RS - acc:  " + str(accuracy_score(y_pred,y_true)))

In [6]:
X_pool = X_trainset
y_pool = y_trainset

In [7]:
#train_idx = idx
pool_idx = np.setdiff1d(range(len(X_pool)),idx)

random_acc_list = []
y_true = [ elem.argmax() for elem in y_test]

stop_training = len(X_pool)+1
for index in range(train_size,stop_training,step):  
    
    if index == train_size:
        train_idx = idx
    else:
        batch_idx = np.random.choice(pool_idx, size=step, replace=False)
        train_idx = np.append(train_idx, batch_idx)

        print("batch_idx = {0}".format(batch_idx.shape))
    print("pool_idx = {0}".format(pool_idx.shape))
    print("train_idx = {0}".format(train_idx.shape))
    
    history = model.fit(X_pool[train_idx], y_pool[train_idx], 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    verbose=1, 
                    callbacks=None, validation_split=0.0, 
                    validation_data=None, shuffle=True, 
                    class_weight=None, sample_weight=None, 
                    initial_epoch=0, steps_per_epoch=None,
                    validation_steps=None)
    
    pool_idx = np.setdiff1d(pool_idx, train_idx)

    pred = model.predict(x=X_test)
    y_pred = [ elem.argmax() for elem in pred]
    acc1= accuracy_score(y_pred,y_true)
    
    random_acc_list.append(acc1)
    print('Accuracy after query {n}: {acc:0.4f}'.format(n=index , acc=acc1))

pool_idx = (4000,)
train_idx = (2000,)
Epoch 1/1
2000/2000 [==============================] - 16s 8ms/step - loss: 2.2999 - acc: 0.1235
Accuracy after query 2000: 0.6300
batch_idx = (200,)
pool_idx = (4000,)
train_idx = (2200,)
Epoch 1/1
2200/2200 [==============================] - 18s 8ms/step - loss: 2.1917 - acc: 0.3750
Accuracy after query 2200: 0.6800
batch_idx = (200,)
pool_idx = (3800,)
train_idx = (2400,)
Epoch 1/1
2400/2400 [==============================] - 18s 7ms/step - loss: 2.0098 - acc: 0.4679
Accuracy after query 2400: 0.6900
batch_idx = (200,)
pool_idx = (3600,)
train_idx = (2600,)
Epoch 1/1
2600/2600 [==============================] - 23s 9ms/step - loss: 1.7516 - acc: 0.5400
Accuracy after query 2600: 0.5700
batch_idx = (200,)
pool_idx = (3400,)
train_idx = (2800,)
Epoch 1/1
2800/2800 [==============================] - 22s 8ms/step - loss: 1.6559 - acc: 0.4679
Accuracy after query 2800: 0.4200
batch_idx = (200,)
pool_idx = (3200,)
train_idx = (3000,)
Epoch 1/1
3000/3

In [8]:
random_acc_list

[0.63,
 0.68,
 0.69,
 0.57,
 0.42,
 0.58,
 0.7,
 0.7,
 0.45,
 0.7,
 0.76,
 0.84,
 0.8,
 0.75,
 0.68,
 0.79,
 0.88,
 0.78,
 0.89,
 0.8,
 0.91]

In [9]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.figure(figsize=(15,8))
sns.pointplot(x=range(train_size,stop_training,step), y=random_acc_list[:20],markers="o",linestyles="-")
plt.grid(True)


KeyError: 2000